practic 1

In [ ]:
#Нод чисел a и b
function gcd(a,b)
    while b != 0
        a, b = b, a % b 
    end
    return abs(a)
end

print(gcd(14,35))

In [ ]:
function extended_gcd(a::T,b::T) where T<:Integer 
    u, v = one(T), zero(T); u1, v1 = 0, 1
    while b != 0        # Инвариант
        r, k = a % b, div(a, b)
        a, b = b, r
        u, v, u1, v1 = u1, v1, u - k * u1, v - k * v1
    end
    if a < 0
        a, u, v = -a, -u, -v
    end
    return a, u, v
end

print(extended_gcd(14,29))

"""
d = um - vn

Инвариант I: НОД(a,b) = НОД(m,n)
a =  um  +  vn
b = u1m +   v1n

Начальные значения этих переменных обеспечивают  выполнения инварианта:
a = m,  b = n
u = 1   v = 0
u1 = 0  v1 = 1

b = 0 условие завершения цикла

r = a - qb = (um - vn) - k(u1m - v1n) = (u - qu1)m + (v - kv1)n
"""

In [ ]:
function invmod(a::T, M::T) where T
    a, x, y = extended_gcd(a, M)
    if x == 1
        return nothing
    end
    return rem(x,M)
end

print(invmod(4,7))

In [ ]:
struct ModuloRingElement{T<:Integer}
    value::T
    modulus::T
#Взятие по модулю M
    function ModuloRingElement(value::T, modulus::T) where {T<:Integer}
        return new(value % modulus, modulus)
    end
end

import Base: +, -, *, ^, inv, -, show

#Операция сложения
+(a::ModuloRingElement, b::ModuloRingElement) = ModuloRingElement(a.value + b.value, a.modulus)

#Операция вычитания
-(a::ModuloRingElement, b::ModuloRingElement) = ModuloRingElement(a.value - b.value, a.modulus)

#Унарный минус
-(a::ModuloRingElement) = ModuloRingElement(-a.value, a.modulus)

#Операция умножения
*(a::ModuloRingElement, b::ModuloRingElement) = ModuloRingElement(a.value * b.value, a.modulus)

#Обращает обратимые элементы
inv(a::ModuloRingElement) = ModuloRingElement(invmod(a.value, a.modulus), a.modulus)

#Вывод в REPL
show(io::IO, a::ModuloRingElement) = print(io, "$(a.value) mod $(a.modulus)")

In [ ]:
struct Polynom{T}
    coeffs::Vector{T}

    function Polynom(coeffs::Vector{T}) where {T<:Number}
        while length(coeffs) > 1 && coeffs[end] == zero(T)
            pop!(coeffs)
        end
        return new{T}(coeffs)
    end
end

#p = Polynom([1, 2, 3])  # создание многочлена 1 + 2x + 3x^2

import Base: +, -, *, show

#Операция сложения
+(a::Polynom{T}, b::Polynom{T}) where {T<:Number} = Polynom{T}(a.coeffs + b.coeffs)

#Операция вычитания
-(a::Polynom{T}, b::Polynom{T}) where {T<:Number} = Polynom{T}(a.coeffs - b.coeffs)

#Унарный минус
-(a::Polynom{T}) where {T<:Number} = Polynom{T}(-a.coeffs)

practic 2

In [ ]:
function myfastpow(a, n)
    k=n
    p=a
    t=1
 
    while k>0
        if iseven(k)
            k /= 2 
            p *= p
        else
            k -= 1 
            t *= p 
        end
    end
    return t
end

print(myfastpow(5,14))

In [ ]:
function FibMat(n)
    m = [1 1; 1 0]
    m = myfastpow(m,n)
    return m[1,1]
end

print(FibMat(9))

In [ ]:
# log[1/a] x = - log[a] x
function log(a0, x, epsilon)
    a = a0; flag  = false
    if a < 1.0
        a = 1.0 / a
        flag = true
    end
    y = 0.0; z = x; t = 1.0;
    while abs(t) > epsilon || z < 1.0 / a || z > a
        # инвариант a^y * z^t == x
        if z > a
            z /= a
            y += t
        elseif  z < 1.0 / a
            z *= a
            y -= t
        else
            z *= z
            t /= 2.0
        end
    end

    return (flag) ? -y : y
end

print(log(0.2,15.67,1e-8))

In [ ]:
function bisection(f::Function, a, b, epsilon)
    f_a = f(a)
    while (b - a > epsilon)
        t = (a + b)/2
        f_t = f(t)
        if (f_t == 0)
            return t
        elseif (f_t * f_a < 0)
            b = t
        else
            a, f_a = t, f_t
        end
    end
    return (a+b) / 2
end

#f(x)=0
function f(x)
    return x * x - 3 * x
end

print(bisection(f, -1, 2, 0.01))

In [ ]:
function borisenco_bisection(f::Function, a, b, epsilon)
    x0 = a; x1 = b; 
    @assert f(x0) * f(x1) <= 0
    while abs(x0 - x1) > epsilon
        c = (x0 + x1) / 2   # Середина отрезка
        if f(x0) * f(c) < 0
            x1 = c
        else
            @assert f(c) * f(x1) <= 0
            x0 = c
        end
    end
    return (x0 + x1) / 2
end

function F(x)
    return cos(x) - x
end


print(borisenco_bisection(F,-1,3.5,1e-8))

In [ ]:
function newton(r::Function, x, epsilon; num_max = 10)
    dx = -r(x)
    k=0
    while abs(dx) > epsilon && k <= num_max
        x += dx
        k += 1
    end
    k > num_max && @warn("Требуемая точность не достигнута")
    return x
end

f(x) = cos(x) - x   

r(x) = f(x)/(sin(x)+1)     #r(x) = f(x)/f'(x)

println(newton(f,1,1))

In [ ]:
struct Polynomials{k}
    coeffs::Vector{k}
    Polynomials{k}(coeffs::Vector{k})  where k = new(coeffs)
end

function Gorner(n::Int, a::T, t) where T
    i = 2
    while (i <= n)
        a[i] = a[i] + a[i-1]*t 
        i = i + 1;
    end
    pop!(a)
    return a
end

p(x) = 6*x^5 - 23*x^4 + 12*x^2 + 86

rp(x) = p(x) / (30*x^4 - 92*x^3 + 24*x)

println(newton(x->x+0,1,1))
println(Gorner(3, [1,2,1], -1))

"""
метод Горнера для вычисления значения многочлена x^2 + 2x + 1 в точке -1. 
"""

In [ ]:
using GLMakie
using Colors

function kelly_fractal(iterations::Int, width::Int, height::Int)
    # Инициализация изображения
    img = zeros(RGBA{Float32}, height, width)

    # Параметры фрактала
    a, b, c, d = -0.8, 0.156, -0.156, 0.8
    e, f = 0.0, 0.0

    # Цикл по пикселям изображения
    for y in 1:height
        for x in 1:width
            # Нормализация координат
            xn = 2.0 * (x / width - 0.5)
            yn = 2.0 * (y / height - 0.5)

            x0, y0 = xn, yn

            # Итерации фрактала
            for _ in 1:iterations
                x1 = a * xn + b * yn + e
                y1 = c * xn + d * yn + f
                xn, yn = x1, y1
            end

            # Определение цвета пикселя
            distance = sqrt((xn - x0)^2 + (yn - y0)^2)
            color = distance < 0.1 ? RGBA(0, 0, 0, 1) : RGBA(1, 1, 1, 1)
            img[y, x] = color
        end
    end

    return img
end

# Параметры фрактала
iterations = 1000
width, height = 800, 800

# Создание фрактала
fractal = kelly_fractal(iterations, width, height)

# Отображение фрактала
figure = Figure(resolution = (width, height))
ax = Axis(figure, aspect = DataAspect())
image!(ax, fractal)
figure

practic 3

In [ ]:
function isPrime(n)

    for i in 2:sqrt(n)
        if n % i == 0
            return false
        end
    end
    return true
end

print(isPrime(103))

In [ ]:
function resheto(n::Integer)
    prime_indexes = ones(Bool, n)
    prime_indexes[begin] = false
    i = 2
    prime_indexes[i^2:i:n] .= false 
    i = 3
   
    while i <= n
        prime_indexes[i^2:2i:n] .= false
        i+=1
        while i <= n && prime_indexes[i] == false
            i+=1
        end

    end
    return findall(prime_indexes)
end

print(resheto(200))

In [ ]:
function factorize(n::IntType) where IntType <: Integer
    list = NamedTuple{(:div, :deg), Tuple{IntType, IntType}}[]
    for p in resheto(Int(ceil(n/2)))
        k = degree(n, p) 
        if k > 0
            push!(list, (div=p, deg=k))
        end
    end
    return list
end

function degree(n, p) 
    k=0
    n, r = divrem(n,p)
    while n > 0 && r == 0
        k += 1
        n, r = divrem(n,p)
    end
    return k
end

print(factorize(420))

In [ ]:
struct Node
    index :: Int
    children :: Vector{Union{Nothing,Node}}
end

function convert!( arr :: Vector, tree :: Dict{Int,Vector})
    isempty(arr) && return

    list = []

    for subarr in arr[1:end-1]
        if isempty(subarr)
            push!(list,nothing)
            continue
        end
        if typeof(subarr) <: Int
            push!(list,subarr)
            continue
        end
        push!(list,subarr[end])
        convert!(subarr,tree)
    end

    tree[arr[end]] = list

    return tree
end

function convert!(tree :: Dict{Int,Vector}; root ::  Union{Int,Nothing}) :: Union{Vector,Int}
    arr = []
    isnothing(root) && return []
    !(root in keys(tree)) && return root
    for subroot in tree[root]
        push!(arr,convert!(tree; root = subroot))
    end
    push!(arr,root)
    return arr
end

function convert!( tree :: Dict{Int,Vector}, root :: Union{Int,Nothing}) ::Union{Node,Nothing}

    isnothing(root) && return nothing
    !(root in keys(tree)) && return Node(root,[])
    node = Node(root,[])

    for sub_root in tree[root]
        push!(node.children, convert!(tree, sub_root))
    end

    return node
end

function convert!( node :: Node) :: Union{Vector,Int}
    arr = []
    length(node.children)==0 && return node.index
    for child in node.children
        if isnothing(child)
            push!(arr, [])
            continue
        end
        push!(arr,convert!(child))
    end
    push!(arr,node.index)
    return arr

end
function convert!(node :: Node, tree :: Dict{Int, Vector}) :: Union{Dict{Int,Vector},Int}
    list = []
    for child in node.children
        if isnothing(child)
            push!(list, nothing)
            continue
        end
        push!(list,child.index)
        length(child.children) != 0 && convert!(child,tree)
    end
    tree[node.index] = list
    return tree
end

In [ ]:
struct VectorTree
    data::Vector{Vector{Int}}
end

function treeHeight(tree::VectorTree, node::Int, h::Int)
    hi = h+1
    if(tree.data[node]!=[])
        for i in 1:length(tree.data[node])
            h = max(h, treeHeight(tree, tree.data[node][i], hi))
        end
        hi = h
    end
    return hi
end

function treeVal(tree::VectorTree)
    v = 0
    for i in 1:length(tree.data)
        v = max(v,length(tree.data[i]))
    end
    return v
end

function treeLeafes(tree::VectorTree, node::Int, l::T) where T
    if(tree.data[node]!=[])
        for i in 1:length(tree.data[node])
            treeLeafes(tree, tree.data[node][i], l)
        end
    else
        l.x+=1
    end
end

function treeNodes(tree::VectorTree, node::Int, n::T) where T
    if(tree.data[node]!=[])
        n.x+=1
        for i in 1:length(tree.data[node])
            treeNodes(tree, tree.data[node][i], n)
        end
    end
end

function treeMidHeight(tree::VectorTree, node::Int, h::Int, heights::T) where T
    h+=1
    if(tree.data[node]!=[])
        for i in 1:length(tree.data[node])
            treeMidHeight(tree, tree.data[node][i], h, heights)
        end
    else
        pushfirst!(heights.x, h)
    end
end

practic 4

In [ ]:
function taylor_exp(n::Int64, x::T) where T
    a0 = one(T)
    res = a0
    for i in 1:n-1
        a0 *= x / (i+1)
        res += a0
    end
    return res

end

println( taylor_exp(20, 0.5))

In [ ]:
function machine_precision()
    eps = Float64(0.5)^52
    while (1.0 + eps) > 1.0
        eps /= 2.0
    end
    return eps
end

println(machine_precision())

In [ ]:
using Plots
function bessel(M::Integer, x::Real)
    sqrx = x*x
    a = 1/factorial(M)
    m = 1
    s = 0 
    
    while s + a != s
        s += a
        a = -a * sqrx /(m*(M+m)*4)
        m += 1
    end
    
    return s*(x/2)^M
end

values = 0:0.1:20
myPlot = plot()
for m in 0:5
	plot!(myPlot, values, bessel.(m, values))
end
display(myPlot)

In [ ]:
using LinearAlgebra
function gaussian_elimination(A::AbstractMatrix{T}, b::AbstractVector{T})::AbstractVector{T} where T
    @assert size(A, 1) == size(A, 2)
    n = size(A, 1) 
    x = zeros(T, n)

    for i in n:-1:1
        x[i] = b[i]
        for j in i+1:n
            x[i] =fma(-x[j] ,A[i,j] , x[i])
        end
        x[i] /= A[i,i]
    end
    return x
end

In [ ]:
function TransformToSteps!(matrix::AbstractMatrix, epsilon::Real = 1e-7)::AbstractMatrix
	@inbounds for k ∈ 1:size(matrix, 1)
		absval, Δk = findmax(abs, @view(matrix[k:end,k]))

		(absval <= epsilon) && throw("Вырожденая матрица")

		Δk > 1 && swap!(@view(matrix[k,k:end]), @view(matrix[k+Δk-1,k:end]))

		for i ∈ k+1:size(matrix,1)
			t = matrix[i,k]/matrix[k,k]
			@. @views matrix[i,k:end] = matrix[i,k:end] - t * matrix[k,k:end]  
		end
	end
	return matrix
end

In [ ]:
@inline function sumprod(vec1::AbstractVector{T}, vec2::AbstractVector{T})::T where T
	s = zero(T)
	@inbounds for i in eachindex(vec1)
	s = fma(vec1[i], vec2[i], s) # fma(x, y, z) вычисляет выражение x*y+z
	end
	return s
end

function ReverseGauss!(matrix::AbstractMatrix{T}, vec::AbstractVector{T})::AbstractVector{T} where T
	x = similar(vec)
	N = size(matrix, 1)

	for k in 0:N-1
		x[N-k] = (vec[N-k] - sumprod(@view(matrix[N-k,N-k+1:end]), @view(x[N-k+1:end]))) / matrix[N-k,N-k]
	end

	return x
end

In [ ]:
for n in 50:50:1000
	println("Матрица порядка ",n,"×",n,":")
	@time ReverseGauss_first!(randn(n,n),randn(n))
	@time ReverseGauss!(randn(n,n),randn(n))
end

ptactic 5

In [ ]:
function insert_sort!(array::AbstractVector{T})::AbstractVector{T} where T <: Number
    n = 1
    # Инвариант: срез array[1:n] - отсортирован

    while n < length(array) 
        n += 1
        i = n

        while i > 1 && array[i-1] > array[i]
            array[i], array[i-1] = array[i-1], array[i]
            i -= 1
        end

        # Утверждение: array[1] <= ... <= array[n]
    end

    return array
end

insert_sort(array::AbstractVector)::AbstractVector = insert_sort!(copy(array))

In [ ]:
function rascheska(arr)
    step = length(arr)
    shrink = 1.247
   while step >=1 
        for i in 1:length(arr) - step
            if arr[i] > arr[i+step]
                arr[i], arr[i+step] = arr[i+step], arr[i]
            end
        end
        step = Int(floor(step/shrink))
   end
    return arr
end

arr = [2, 1, 5, 4, 3, 2, 7, 2, 8, 9]
print(rascheska(arr))

In [ ]:
function compare!(arr, i, g)
    if i < 0 || g < 0 
        return;
    end
    if arr[i] > arr[g]
        arr[i], arr[g] = arr[g], arr[i]
        compare!(arr, i, 2i - g)
    end
end

function shell_sort!(arr)
    len = length(arr)   
    step = div(len, 2)          

    while step >= 1
        for i in 1:len - step       
            compare!(arr,i,i + step)
        end
        step = div(step, 2)
    end
    return arr
end

arr = [1, 1, 5, 4, 3, 2, 7, 2, 8, 9]    
print(shell_sort!(arr))

In [ ]:
function Base.merge!(a1, a2, a3)::Nothing 
    i1, i2, i3 = 1, 1, 1
    @inbounds while i1 <= length(a1) && i2 <= length(a2) 
        if a1[i1] < a2[i2]
            a3[i3] = a1[i1]
            i1 += 1
        else
            a3[i3] = a2[i2]
            i2 += 1
        end
        i3 += 1
    end
    @inbounds if i1 > length(a1)
        a3[i3:end] .= @view(a2[i2:end])  
    else
        a3[i3:end] .= @view(a1[i1:end])
    end
    nothing
end



function merge_sort!(a)
    b = similar(a) # - вспомогательный массив того же размера и типа, что и массив a
    N = length(a)
    n = 1 # n - текущая длина блоков
    @inbounds while n < N
        K = div(N,2n) # - число имеющихся пар блоков длины n
        for k in 0:K-1
            merge!(@view(a[(1:n).+k*2n]), @view(a[(n+1:2n).+k*2n]), @view(b[(1:2n).+k*2n]))
        end
        if N - K*2n > n # - осталось еще смержить блок длины n и более короткий остаток
            merge!(@view(a[(1:n).+K*2n]), @view(a[K*2n+n+1:end]), @view(b[K*2n+1:end]))
        elseif 0 < N - K*2n <= n # - оставшуюся короткую часть мержить не с чем
            b[K*2n+1:end] .= @view(a[K*2n+1:end])
        end
        a, b = b, a
        n *= 2
    end
    if isodd(log2(n)) # - если цикл был выполнен нечетное число раз, то b - это исходная ссылка на массив (на внешний массив), и a - это ссылка на вспомогательный массив (локальный)
        b .= a # b = copy(a) - это было бы не то же самое, т.к. при этом получилась бы ссылка на новый массив, который создает функция copy
        a = b
    end
    return a # - исходная ссылка на внешний массив (проверить, что это так, можно с помощью ===)
end

In [ ]:
function calc_sort!(array::AbstractVector{T})::AbstractVector{T} where T <: Number
    min_val, max_val = extrema(array)
    num_val = zeros(T, max_val - min_val + 1) # - число всех возможных значений

    for val in array
        num_val[val-min_val+1] += 1
    end  
    k = 0

    for (i, num) in enumerate(num_val)
        array[k+1:k+num] .= min_val+i-1
        k += num
    end

    return array
end

calc_sort(array::AbstractVector)::AbstractVector = calc_sort!(copy(array))

# Порядковые статистики, алгоритм быстрого вычисления порядковых статистик
function order_statistics!(array::AbstractVector{T}, index::Integer)::T where T
	function part_sort!(indexes_range::AbstractUnitRange, b)
		K, L, M = indexes_range[1]-1, indexes_range[begin]-1, indexes_range[end] # 0, 0, N
		#ИНВАРИАНТ: array[indexes_range[begin]:K] < b && array[K+1:L] == b && array[M+1:indexes_range[end]] > b
		while L < M 
			if array[L+1] == b
				L += 1
			elseif array[L+1] > b
				array[L+1], array[M] = array[M], array[L+1]
				M -= 1
			else # if array[L+1] < b
				L += 1; K += 1
				array[L], array[K] = array[K], array[L]
			end
		end    
		return indexes_range[begin]:K, M+1:indexes_range[end] 
		# - эти диапазоны индексов определяют ещё не отсортированные части массива array
	end

	function find(indexes_range)
		left_range, right_range = part_sort!(indexes_range, array[rand(indexes_range)]) 
		# - здесь "базовый" элемент массива выбирается случайным образом
		if index in left_range
			return find(left_range) 
		elseif index in right_range
			return find(right_range)
		else
			return array[index]
		end
	end

	find(firstindex(array):lastindex(array))
end

order_statistics(array::AbstractVector, index::Integer) = order_statistics!(copy(array), index)

# Реализация кучи на базе массива O(n)
function heap!(array::AbstractVector{T})::AbstractVector{T} where T <: Number
    N = length(array)

    for i in 1:N÷2
        if array[i] < array[2i]
            array[i], array[2i] = array[2i], array[i]
        end
        
        if 2i+1 <= N && array[i] < array[2i+1]
            array[i], array[2i+1] = array[2i+1], array[i]
        end
    end

    return array
end

heap(array::AbstractVector)::AbstractVector = heap!(copy(array))

practic 6

In [ ]:
using LinearAlgebra

Vector2D{T<:Real} = NamedTuple{(:x, :y), Tuple{T,T}}

# Сложение векторов
Base. +(a::Vector2D{T},b::Vector2D{T}) where T = Vector2D{T}(Tuple(a) .+ Tuple(b))

# Разность векторов
Base. -(a::Vector2D{T}, b::Vector2D{T}) where T = Vector2D{T}(Tuple(a) .- Tuple(b))

# Векторное произведение векторов
Base. *(α::T, a::Vector2D{T}) where T = Vector2D{T}(α.*Tuple(a))

# Длина вектора
LinearAlgebra.norm(a::Vector2D) = norm(Tuple(a))

# Скалярное произведение векторов (a,b)=|a||b|cos(a,b)
LinearAlgebra.dot(a::Vector2D{T}, b::Vector2D{T}) where T = dot(Tuple(a), Tuple(b))

# Косое произведение    (a,b)=|a||b|sin(a,b)
xdot(a::Vector2D{T}, b::Vector2D{T}) where T = a.x*b.y-a.y*b.x

# Синус угла между векторами
Base.sin(a::Vector2D{T}, b::Vector2D{T}) where T = xdot(a,b)/norm(a)/norm(b)

# Косинус угла между векторами
Base. cos(a::Vector2D{T}, b::Vector2D{T}) where T = dot(a,b)/norm(a)/norm(b)

# Угол между векторами
Base.angle(a::Vector2D{T}, b::Vector2D{T}) where T = atan(sin(a,b),cos(a,b))

# Знак угла
Base.sign(a::Vector2D{T}, b::Vector2D{T}) where T = sign(sin(a,b))

# Отрезок 
Segment2D{T<:Real} = NamedTuple{(:A, :B), NTuple{2,Vector2D{T}}}

In [ ]:
function is_one(P::Vector2D{T}, Q::Vector2D{T}, s::Segment2D{T}) where T 
    l = s.B-s.A     # Направляющий вектор заданной прямой
    return sin(l, P-s.A)*sin(l,Q-s.A) > 0   # Если угол между (l,AP) и (l,BP) имеют один знак, значит точки с 1 стороны 
end

In [ ]:
# Общий случай для заданной кривой F(x,y) = 0
is_one_area(F::Function, P::Vector2D{T}, Q::Vector2D{T}) where T = (F(P...)*F(Q...)>0)

In [ ]:
function intersection(s1::Segment2D{T},s2::Segment2D{T}) where T
    A = [s1.B[2]-s1.A[2]  s1.A[1]-s1.B[1]
         s2.B[2]-s2.A[2]  s2.A[1]-s2.B[1]]

    b = [s1.A[2]*(s1.A[1]-s1.B[1]) + s1.A[1]*(s1.B[2]-s1.A[2])
         s2.A[2]*(s2.A[1]-s2.B[1]) + s2.A[1]*(s2.B[2]-s2.A[2])]

    x,y = A\b   # Случай с определителем матрицы отбросим
    # Если точка не принадлежит какому-либо сегменту, то нет ответа
    if isinner((;x, y), s1)==false || isinner((;x, y), s2)==false   
        return nothing
    end
    return (;x, y) #Vector2D{T}((x,y))
end

# Проверка на то, является ли точка внутренней для сегмента
isinner(P::Vector2D, s::Segment2D) = (s.A.x <= P.x <= s.B.x || s.A.x >= P.x >= s.B.x)  && (s.A.y <= P.y <= s.B.y || s.A.y >= P.y >= s.B.y)

In [ ]:
function isinside(point::Vector2D{T},polygon::AbstractArray{Vector2D{T}})::Bool where T
	sum = zero(Float64)

    # Если сумма углов 0 — снаружи. 2π — внутри
	for i in firstindex(polygon):lastindex(polygon)
		sum += angle( polygon[i] - point , polygon[i % lastindex(polygon) + 1] - point )
	end
	
    # Чтобы не было неточностей при сравнении сравним с π(если < то это 0, если больше то 2π)
	return abs(sum) > π
end

In [ ]:
function isconvex(polygon::AbstractArray{Vector2D{T}})::Bool where T
	for i in firstindex(polygon):lastindex(polygon) # Проверяем поочерёдно углы(если все <180 выпуклый)
		if angle( polygon[i > firstindex(polygon) ? i - 1 : lastindex(polygon)] - polygon[i] , polygon[i % lastindex(polygon) + 1] - polygon[i] ) >= π
			return false
		end
	end
	return true
end

In [ ]:
function jarvis!(points::AbstractArray{Vector2D{T}})::AbstractArray{Vector2D{T}} where T

    function next!(convex_shell2::AbstractVector{Int64}, points2::AbstractVector{Vector2D{T}}, ort_base::Vector2D{T})::Int64 where T
        cos_max = typemin(T)
        i_base = convex_shell2[end]
        resize!(convex_shell2, length(convex_shell2) + 1)
        for i in eachindex(points2)
            if points2[i] == points2[i_base] # тут не обязательно, что i == i_base
                continue
            end
            ort_i = points2[i] - points2[i_base] # - не нулевой вектор, задающий направление на очередную точку
            cos_i = cos(ort_base, ort_i)
            if cos_i > cos_max
                cos_max = cos_i
                convex_shell2[end] = i
            elseif cos_i == cos_max && dot(ort_i, ort_i) > dot(ort_base, ort_base) # на луче, содержащем сторону выпуклого многоугольника, может оказаться более двух точек заданного множества (надо выбрать самую дальнюю из них)
                convex_shell2[end] = i
            end
        end
        return convex_shell2[end]
    end

    @assert length(points) > 1
    ydata = [points[i].y for i in firstindex(points):lastindex(points)]
    i_start = findmin(ydata)
    convex_shell = [i_start[2]]
    ort_base = (x=oneunit(T), y=zero(T))

    while next!(convex_shell, points, ort_base) != i_start[2]
        ort_base = points[convex_shell[end]] - points[convex_shell[end-1]]
    end

	pop!(convex_shell)

    return points[convex_shell]
end

println("Алгоритм Джарвиса: ", jarvis!( [
		(x=0.0,y=0.0),
		(x=5.0,y=1.0),
		(x=4.0,y=3.0),
		(x=1.0,y=9.0),
		(x=-3.0,y=8.0),
		(x=-5.0,y=2.0),
		(x=-2.0,y=3.0),
	] ) )

# 8. Написать функцию, реализующую алгоритм Грехома построения выпуклой оболочки заданных точек плоскости.
function grekhom!(points::AbstractArray{Vector2D{T}})::AbstractArray{Vector2D{T}} where T
	ydata = (points[i].y for i in firstindex(points):lastindex(points))

    i_start = findmin(ydata)

    points[begin], points[i_start[2]] = points[i_start[2]], points[begin]

    sort!(@view(points[begin + 1:end]), by=(point -> angle(point, (x=oneunit(T), y=zero(T)))))

    push!(points, points[begin])

    convex = [firstindex(points), firstindex(points) + 1, firstindex(points) + 2]

    for i in firstindex(points)+3:lastindex(points)
        while length(convex) > 1 && sign(points[i] - points[convex[end-2]], points[convex[end-1]] - points[convex[end-2]]) < 0
            pop!(convex)
        end

        push!(convex, i)
    end

   	pop!(convex)

    return points[convex]
end


grekhom!(points::AbstractArray{Vector2D}) = jarvis!(points::AbstractArray{Vector2D})

In [ ]:
function area_trapeze(poly::AbstractArray{Vector2D{T}})::T where T
    res = zero(T)

	# area = (yk + yk+1)(xk+1 − xk)/2
    for i in firstindex(poly):lastindex(poly)-1
        res += (poly[i].y + poly[i+1].y) * (poly[i+1].x - poly[i].x) / 2
    end

    return res
end

In [ ]:
practic 7

In [ ]:
function next_repit_plasement!(p::Vector{T}, n::T) where T<:Integer
    i = findlast(x->(x < n), p)
    isnothing(i) && (return nothing)
    p[i] += 1
    p[i+1:end] .= 1 
    return p
end

In [ ]:
function next_permutation!(p::AbstractVector)
    n = length(p)
    k = 0 
    for i in reverse(1:n-1) 
        reverse(firstindex(p):lastindex(p)-1)
        if p[i] < p[i+1]
             k=i
             break
         end
    end

    k == firstindex(p)-1 && return nothing 
    i=k+1
    while i<n && p[i+1]>p[k] 
        p[k]
        i += 1
    end

    p[k], p[i] = p[i], p[k]
    reverse!(@view p[k+1:end])
    return p

end

In [ ]:
function next_indicator!(indicator::AbstractVector{Bool})
    i = findlast(x->(x==0), indicator)
    isnothing(i) && return nothing
    indicator[i] = 1
    indicator[i+1:end] .= 0
    return indicator 
end

In [ ]:
function next_indicator!(indicator::AbstractVector{Bool}, k)
    @assert(k <= n)
    i = lastindex(indicator)
    while indicator[i] == 0
        i -= 1
    end
    m = 0 
    while i >= firstindex(indicator) && indicator[i] == 1 
        m += 1
        i -= 1
    end
    if i < firstindex(indicator)
        return nothing
    end
    indicator[i] = 1
    indicator[i + 1:end] .= 0
    indicator[lastindex(indicator) - m + 2:end] .= 1
    return indicator 
end

In [ ]:
function next_split!(s ::AbstractVector{Int64}, k)
    k == 1 && return (nothing, 0)
    i = k-1 
    while i > 1 && s[i-1]>=s[i]
        i -= 1
    end
    s[i] += 1
    r = sum(@view(s[i+1:k]))
    k = i+r-1
    s[(i+1):(length(s)-k)] .= 1
    return s, k
end

In [ ]:
abstract type AbstractCombinObject
    # value::Vector{Int} - это поле предполагается у всех конкретных типов, наследующих от данного типа
end

Base.iterate(obj::AbstractCombinObject) = (get(obj), nothing)
Base.iterate(obj::AbstractCombinObject, state) = (isnothing(next!(obj)) ? nothing : (get(obj), nothing))
 
# Размещения с повторениями
struct RepitPlacement{N,K} <: AbstractCombinObject
    value::Vector{Int}
    RepitPlacement{N,K}() where {N, K} = new(ones(Int, K))
end
 
Base.get(p::RepitPlacement) = p.value
next!(p::RepitPlacement{N,K}) where {N, K} = next_repit_placement!(p.value, N)

println("Размещения с повторениями")
for a in RepitPlacement{2,3}() 
    println(a)
end
 
# Cтруктура для представления перестановок
struct Permute{N} <: AbstractCombinObject
    value::Vector{Int}
    Permute{N}() where N = new(collect(1:N))
end
 
Base.get(obj::Permute) = obj.value
next!(permute::Permute) = next_permute!(permute.value)
 
println("Перестановки")
for p in Permute{4}()
    println(p)
end

# Все подмножества
struct Subsets{N} <: AbstractCombinObject
    indicator::Vector{Bool}
    Subsets{N}() where N = new(zeros(Bool, N))
end
 
Base.get(sub::Subsets) = sub.indicator
next!(sub::Subsets) = next_indicator!(sub.indicator) 
 
println("Все подмножества N-элементного множества")
for sub in Subsets{4}()
    println(sub)
end

# k-элементные подмоножества 
struct KSubsets{M,K} <: AbstractCombinObject
    indicator::Vector{Bool}
    KSubsets{M, K}() where{M, K} = new([zeros(Bool, length(M)-K); ones(Bool, K)])
end
 
Base.get(sub::KSubsets) = sub.indicator
next!(sub::KSubsets{M, K}) where{M, K} = next_indicator!(sub.indicator, K) 
 
for sub in KSubsets{1:6, 3}()
    sub |> println
end
 
# Разбиения
mutable struct NSplit{N} <: AbstractCombinObject
    value::Vector{Int}
    num_terms::Int # число слагаемых (это число мы обозначали - k)
    NSplit{N}() where N = new(vec(ones(Int, N)), N)
end
 
Base.get(nsplit::NSplit) = nsplit.value[begin:nsplit.num_terms]
function next!(nsplit::NSplit)
    a, b = next_split!(nsplit.value, nsplit.num_terms)
    if isnothing(a) return nothing end
    nsplit.value, nsplit.num_terms = a, b
    get(nsplit)
end
 
println("Разбиения")
for s in NSplit{5}()
    println(s)
end
 

In [ ]:
function dfs(graph, start, visited)
    visited[start] = true        # Помечаем текущую вершину как посещенную
    for v in graph[start]        # Обходим все смежные вершины
        if !visited[v]           # Если вершина еще не была посещена, вызываем функцию dfs для нее
            dfs(graph, v, visited)
        end
    end
end

#Функция для проверки связности графа
function is_connected(graph)
    visited = falses(length(graph))     # Создаем массив для хранения информации о посещенных вершинах
    dfs(graph, 1, visited)              # Выбираем произвольную вершину и вызываем функцию dfs для нее
    return all(visited)                 # Если все вершины были посещены, то граф связный
end

# Создаем граф в виде массива списков смежности
graph = [[2, 3], [1, 3], [1, 2, 4], [3]]

# Проверяем граф на связность
is_connected(graph)

In [ ]:
function dfs(graph, start, visited, component)
    visited[start] = true             # Помечаем текущую вершину как посещенную
    push!(component, start)           # Добавляем текущую вершину в текущую компоненту связности
    for v in graph[start]             # Обходим все смежные вершины
        if !visited[v]                # Если вершина еще не была посещена, вызываем функцию dfs для нее
            dfs(graph, v, visited, component)
        end
    end
end

# Функция для нахождения компонент связности графа
function find_components(graph)
    visited = falses(length(graph))     # Создаем массив для хранения информации о посещенных вершинах
    components = []                     # Создаем массив для хранения компонент связности
    for v in 1:length(graph)            # Обходим все вершины графа
        if !visited[v]                  # Если вершина еще не была посещена, создаем новую компоненту связности и вызываем функцию dfs для нее
            component = []
            dfs(graph, v, visited, component)
            push!(components, component)          # Добавляем найденную компоненту связности в массив компонент
        end
    end
    return components         # Возвращаем массив компонент связности
end

# Создаем граф в виде массива списков смежности
graph = [[2, 3], [1, 3], [1, 2, 4], [3], [6], [5, 7], [6]]
components = find_components(graph)     # Находим компоненты связности графа
for component in components             # Выводим найденные компоненты связности на экран
    println(component)
end

In [ ]:
using LightGraphs

function is_bipartite(g)
    colors = fill(0, nv(g))          # Создаем массив цветов для каждой вершины графа
    
    # Функция для проверки двудольности графа
    function dfs(v, color)
        colors[v] = color
        for w in neighbors(g, v)
            if colors[w] == color
                return false
            elseif colors[w] == 0 && !dfs(w, -color)
                return false
            end
        end
        return true
    end
    
    # Проверяем каждую вершину графа
    for v in vertices(g)
        if colors[v] == 0 && !dfs(v, 1)
            return false
        end
    end
    
    return true
end

g = SimpleGraph(4)
add_edge!(g, 1, 2)
add_edge!(g, 2, 3)
add_edge!(g, 3, 4)
add_edge!(g, 4, 1)

if is_bipartite(g)
    println("Граф является двудольным")
else
    println("Граф не является двудольным")
end